> You know, Apple Computers started in a garage, and we are starting in a condo. So we already have a leg up on Apple. - Pam Beesly


Today started as peaceful morning in Scraton Pennsylvania. There has been a long time vacated position ever since Ryan was fired as VP.
Everyone from the office just learnt that Charles Miner, previously worked at Saticoy Steel, is joining Dunder Mifflin as new VP.

Let's add Charles to Dunder Mifflin's scim group.


In [ ]:
# First we need to import the scim client and configuration
from sync_app.client import ClientConfiguration, SCIMClient

# scim client configuration require server URL and access token.
# The server URL is the base URL of the SCIM server.
server_url = "https://api.scim.dev/scim/v2/"
# The access token is the token that is used to authenticate the client to the SCIM server.
# You can get the access token from the website
# Visit https://scim.dev/apikey/ to get your token
auth_token = "YOUR_ACCESS_CODE"


# Now we have everything we need to create a scim client configuration
scim_dev_client_config = ClientConfiguration(
    server_url=server_url, auth_token=auth_token
)

# Now we can create a scim client using the configuration
scim_dev_client = SCIMClient(scim_dev_client_config)

In [12]:
# This block of code will generate all the users and update it to remote SCIM server. You should not run this code more than once.
# Otherwise you might get group name already exist error.
# If you want, you could run this to delete the group.
#
# scim_dev_client.delete_group("9b806eaa-4455-485e-a4cf-98714934dc5a")
#

from scim_helper import course_setup

# You can ignore this part for now, the crouse_setup function create new group and add bunch exist users to the group.
dunder_mifflin_group_id, user_ids = course_setup(auth_token)

In [14]:
# You can take a look at the group_id and user_ids
print("Group ID is the UUID for the group created by the course_setup helper function")
print(dunder_mifflin_group_id)
print(
    "It also create a nice dictionary of user_id to user_name for the users added to the group."
)
user_ids

Group ID is the UUID for the group created by the course_setup helper function
9b8a54c0-3a94-4c6a-a089-359dd95f9c67
It also create a nice dictionary of user_id to user_name for the users added to the group.


{'Micheal Scott': '9b8a54c1-182b-4462-9678-b880db8cf74a',
 'Jim Halpert': '9b8a54c1-fd71-4def-87cf-ebe6b2271ead',
 'Dwight Schrute': '9b8a54c3-0216-4aa0-87ce-019353b76180',
 'Pam Beesly': '9b8a54c3-f7d7-4096-b07b-83301566dcd2',
 'Ryan Howard': '9b8a54c4-dd1c-48ec-b9eb-c790ec776f02',
 'Andy Bernard': '9b8a54c5-bc62-4f15-9ad3-9cafff178b8d',
 'Kevin Malone': '9b8a54c6-a050-4a65-b1a2-4e18ee1f80cb',
 'Oscar Martinez': '9b8a54c7-84fd-4c87-a7f8-677e2de988e2',
 'Angela Martin': '9b8a54c8-6027-4c6d-9825-18c21454e354',
 'Stanley Hudson': '9b8a54c9-433d-409b-8265-a1011ef7d86a',
 'Phyllis Vance': '9b8a54ca-2ca6-4e41-a7a0-c65ac0513f77',
 'Meredith Palmer': '9b8a54cb-1139-4589-bde8-5c842d36af0e',
 'Creed Bratton': '9b8a54cb-f180-40d5-8c94-fc3f90f6d946',
 'Kelly Kapoor': '9b8a54cc-cf33-408c-91c9-dc1def26c0be',
 'Toby Flenderson': '9b8a54cd-ab55-41a8-b0ff-ca1b4e37a5df'}

Enough of buzy work, now let's create a new user object!

Here is what User object required!

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We can go over them one by one.


The first thing we need is some meta data!
ResourceMeta does provide create_meta function to create THE meta

```
def create_meta(resource_type: ResourceType, location: str) -> "ResourceMeta":
```

It requires the resource type and the resource URL
There are five different type resource that exist

- User
- Group
- ServiceProviderConfig
- ResourceType
- Schema

Looks like the one we need right now is `User` because duh, we are creating a user object.

But what the hack is location?
Resource Location is the endpoint where a resource could be re-fetched from the same service prodiver.
Since we (Duo) is storing the user, the location would be `https://duo.com/Users/<uuid>`


In [16]:
from sync_app.scim.resource import ResourceMeta

# We would also need some uuid
from uuid import uuid5, NAMESPACE_X500

# uuid5 is used to generate a unique identifier from a name and a namespace
# We would use the NAMESPACE_X500 as the namespace
# So we could use DN as the name to generate the uuid

uuid = uuid5(NAMESPACE_X500, "uid=Charles.Miner,DC=dundermifflin,DC=com")

meta = ResourceMeta.create_meta(
    "User",
    "https://duo.local/Users/{}".format(
        str(uuid),
    ),
)

Looking back on User Object

```
    def __init__(
        self,
        meta: ResourceMeta,
        user_id: UUID,
        displayName: Optional[str],
        active: bool,
        userName: str,
        name: Optional[Name],
        emails: Optional[Iterable[Email]],
        groups: Optional[Iterable[ResourceRef]] = None,
    ):
```

We actually killed two bird with one stone since meta require uuid.
`DisplayName` is The name of the User, suitable for display to end-users. The name SHOULD be the full name of the User being described, if known. So it should be like `Charles Miner`
`active` is A Boolean value indicating the User's administrative status. So `True` should be it.
`userName` is Unique identifier for the User, typically used by the user to directly authenticate to the service provider. Each User MUST include a non-empty userName value. This identifier MUST be unique across the service provider's entire set of Users. REQUIRED. So `cminer` should be fine.

`Name` is a werid one, because it contain some sub attribute.
`Name` Object looks like this, it's all optional but we fill familyName and givenName easily.

```
@dataclass
class Name:
    formatted: Optional[str] = None
    familyName: Optional[str] = None
    givenName: Optional[str] = None
```

`emails` is also a object that contain subattribute.

```
@dataclass
class Email:
    value: Optional[str] = None
    type: Optional[str] = None
    primary: Optional[bool] = None
    display: Optional[str] = None
```

`value` is email address
`type` is for work/home/other/etc
`primary` is if the email is the preferred one.
`display` is the the thing make least sense, so we will skip that for now.

`groups` is use set the group for the user, for now we are going to use the exist Dunder Mifflin Group!


In [17]:
# Now let's create a user!
from sync_app.scim.user import User, Name, Email
from sync_app.scim.resource import ResourceRef

cminer = User(
    meta=meta,
    user_id=uuid,
    displayName="Charles.Miner",
    active=True,
    userName="cminer",
    name=Name(formatted="Charles Miner", familyName="Miner", givenName="Charles"),
    emails=[
        Email(
            value="Charles.Miner@dundermifflin.com",
            type="work",
            primary=True,
        )
    ],
    groups=[
        ResourceRef(
            ref="https://api.scim.dev/scim/v2/Group/{}".format(dunder_mifflin_group_id),
            value=dunder_mifflin_group_id,
            display="Dunder Mifflin",
        )
    ],
)

# Now we can try to push the user to the server!
response = scim_dev_client.create_user(cminer)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T19:13:16+00:00',
  'lastModified': '2024-03-11T19:13:16+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a5558-5558-41ce-a20c-f425207f4471'},
 'id': '9b8a5558-5558-41ce-a20c-f425207f4471',
 'active': True,
 'userName': 'cminer',
 'emails': [{'value': 'Charles.Miner@dundermifflin.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'Charles.Miner@dundermifflin.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': 'Charles Miner',
  'familyName': 'Miner',
  'givenName': 'Charles'}}

In [19]:
# We should also keep track of the newly created user to our user_ids dictionary
user_ids["Charles.Miner"] = response.to_dict()["id"]
user_ids

{'Micheal Scott': '9b8a54c1-182b-4462-9678-b880db8cf74a',
 'Jim Halpert': '9b8a54c1-fd71-4def-87cf-ebe6b2271ead',
 'Dwight Schrute': '9b8a54c3-0216-4aa0-87ce-019353b76180',
 'Pam Beesly': '9b8a54c3-f7d7-4096-b07b-83301566dcd2',
 'Ryan Howard': '9b8a54c4-dd1c-48ec-b9eb-c790ec776f02',
 'Andy Bernard': '9b8a54c5-bc62-4f15-9ad3-9cafff178b8d',
 'Kevin Malone': '9b8a54c6-a050-4a65-b1a2-4e18ee1f80cb',
 'Oscar Martinez': '9b8a54c7-84fd-4c87-a7f8-677e2de988e2',
 'Angela Martin': '9b8a54c8-6027-4c6d-9825-18c21454e354',
 'Stanley Hudson': '9b8a54c9-433d-409b-8265-a1011ef7d86a',
 'Phyllis Vance': '9b8a54ca-2ca6-4e41-a7a0-c65ac0513f77',
 'Meredith Palmer': '9b8a54cb-1139-4589-bde8-5c842d36af0e',
 'Creed Bratton': '9b8a54cb-f180-40d5-8c94-fc3f90f6d946',
 'Kelly Kapoor': '9b8a54cc-cf33-408c-91c9-dc1def26c0be',
 'Toby Flenderson': '9b8a54cd-ab55-41a8-b0ff-ca1b4e37a5df',
 'Charles.Miner': '9b8a5558-5558-41ce-a20c-f425207f4471'}

Now we have Charles in the system, Charles the new VP finally able to access his new laptop and all customer information for the first day!


Even Micheal had good time with Charles in the beginning, the tension between Micheal and Charles is so thich you can pratical cut it with a butter knife.

This result Micheal put his 2 week notice by the end of day. Micheal approved multiple Dunder Mifflin employee to join Micheal to his new paper company!


After goofy around for 2 week, the final day at Dunder Mifflin for Micheal has arrived. There is no one willing to quit the job at Dunder Mifflin and join Micheal for his new adventure.


Let's remove Micheal from the Dunder Mifflin group, so he can no long access inportant company secrets.


In [20]:
# Lets first deactive Micheal's status
from sync_app.scim.patchop import PatchOp, Operation

deactive_patch_op = PatchOp(
    operations=[Operation(op="replace", path="active", value=False)]
)

response = scim_dev_client.update_user(
    id=user_ids["Micheal Scott"], ops=deactive_patch_op
)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T19:11:37+00:00',
  'lastModified': '2024-03-11T19:13:39+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a54c1-182b-4462-9678-b880db8cf74a'},
 'id': '9b8a54c1-182b-4462-9678-b880db8cf74a',
 'active': False,
 'userName': 'mscott',
 'emails': [{'value': 'micheal.scott@dundermifflin.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'micheal.scott@dundermifflin.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Scott', 'givenName': 'Micheal'}}

In [21]:
# To remove user from group we need to make a patch call to the group API
# The API take a patch ops json object. So lets make that first
from sync_app.scim.patchop import PatchOp, Operation

remove_patch_op = PatchOp(
    operations=[Operation(op="remove", path="members", value=user_ids["Micheal Scott"])]
)

response = scim_dev_client.update_group(
    id=dunder_mifflin_group_id,
    ops=remove_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T19:11:36+00:00',
  'lastModified': '2024-03-11T19:11:36+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a54c0-3a94-4c6a-a089-359dd95f9c67'},
 'id': '9b8a54c0-3a94-4c6a-a089-359dd95f9c67',
 'displayName': 'Dunder Mifflin',
 'name': 'Dunder Mifflin'}

> Confidence-- it's the food of the wise man, but the liquor of the fool. --Vikram


Oh no, Micheal Scott Paper Company just started.
We need to do some scim work to keep the scim record up to dates.
Here are list of things we need to.

- Create a new group called Micheal Scott Paper Company.
- Add Micheal, Pam, Ryan to the new Group.
- Deactive Pam from Dunder Mifflin group since she is no long working for Dunder Mifflin.


In [23]:
# Similar to before, this block of code is one time use only.
# If you want to run this code again, you should delete the group first.
#
# scim_dev_client.delete_group("9b8a1233-446e-4b74-8693-e76543357fa7")
#

# Create a new group
from sync_app.scim.group import Group
from sync_app.scim.resource import ResourceMeta
from uuid import uuid5, NAMESPACE_X500

micheal_scott_paper_company = Group(
    displayName="Micheal Scott Paper Company",
    group_id=uuid5(NAMESPACE_X500, "cn=MichealScott,DC=PaperCompany,DC=com"),
    meta=ResourceMeta.create_meta(
        "Group",
        "https://duo.local/Groups/{}".format(
            uuid5(NAMESPACE_X500, "cn=MichealScott,DC=PaperCompany,DC=com")
        ),
    ),
)

response = scim_dev_client.create_group(micheal_scott_paper_company)

micheal_scott_paper_company_id = response.to_dict()["id"]

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T19:15:44+00:00',
  'lastModified': '2024-03-11T19:15:44+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a563a-3d17-4f36-8f4c-9c33649a7006'},
 'id': '9b8a563a-3d17-4f36-8f4c-9c33649a7006',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company'}

In [24]:
# Now let's try to do multiple operations in one patch call
# Let's create a patch operation to add all three people to the new group
from sync_app.scim.patchop import PatchOp, Operation

add_three_peope_op = PatchOp(
    operations=[
        Operation(
            op="add",
            path="members",
            value=[
                {"value": user_ids["Micheal Scott"]},
                {"value": user_ids["Pam Beesly"]},
                {"value": user_ids["Ryan Howard"]},
            ],
        )
    ]
)

response = scim_dev_client.update_group(
    id=micheal_scott_paper_company_id,
    ops=add_three_peope_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T19:15:44+00:00',
  'lastModified': '2024-03-11T19:15:44+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a563a-3d17-4f36-8f4c-9c33649a7006'},
 'id': '9b8a563a-3d17-4f36-8f4c-9c33649a7006',
 'displayName': 'Micheal Scott Paper Company',
 'name': 'Micheal Scott Paper Company'}

In [25]:
# Now let's deactive Pam from dunder mifflin group
# We have done the similar thing for Micheal Scott before.
from sync_app.scim.patchop import PatchOp, Operation

remove_patch_op = PatchOp(
    operations=[
        Operation(op="remove", path="members", value=user_ids["Pam Beesly"]),
        Operation(op="remove", path="members", value=user_ids["Ryan Howard"]),
    ]
)

response = scim_dev_client.update_group(
    id=dunder_mifflin_group_id,
    ops=remove_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T19:11:36+00:00',
  'lastModified': '2024-03-11T19:11:36+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a54c0-3a94-4c6a-a089-359dd95f9c67'},
 'id': '9b8a54c0-3a94-4c6a-a089-359dd95f9c67',
 'displayName': 'Dunder Mifflin',
 'name': 'Dunder Mifflin'}

Finally Let's try something new inbound calls.

We already moved all three Micheal Scott Paper Company Employee to their own group. Let's try to use scim filter to get their group information!


In [27]:
# The esiast way to get a group or user is to use the get_user or get_group method along with the id

# For example, this is how you get the original Dunder Mifflin group

response = scim_dev_client.get_group(dunder_mifflin_group_id)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
 'meta': {'resourceType': 'Group',
  'create': '2024-03-11T19:11:36+00:00',
  'lastModified': '2024-03-11T19:11:36+00:00',
  'location': 'https://api.scim.dev/scim/v2/Groups/9b8a54c0-3a94-4c6a-a089-359dd95f9c67'},
 'id': '9b8a54c0-3a94-4c6a-a089-359dd95f9c67',
 'displayName': 'Dunder Mifflin',
 'name': 'Dunder Mifflin'}

In [28]:
# Now let's try to get the user we just created but with filter.

# First we need to create a new filter
from sync_app.client import Filter

filter = Filter(filter='displayName eq "Dunder Mifflin"')
response = scim_dev_client.get_groups(filter=filter)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:api:messages:2.0:ListResponse'],
 'totalResults': 1,
 'Resources': [{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:Group'],
   'meta': {'resourceType': 'Group',
    'create': '2024-03-11T19:11:36+00:00',
    'lastModified': '2024-03-11T19:11:36+00:00',
    'location': 'https://api.scim.dev/scim/v2/Groups/9b8a54c0-3a94-4c6a-a089-359dd95f9c67'},
   'id': '9b8a54c0-3a94-4c6a-a089-359dd95f9c67',
   'displayName': 'Dunder Mifflin',
   'name': 'Dunder Mifflin'}],
 'itemsPerPage': 1,
 'startIndex': 1}

In [29]:
# Let's get all users that under Micheal Scott Paper Company!

from sync_app.client import Filter

filter = Filter(filter='groups.value eq "{}"'.format(micheal_scott_paper_company_id))
response = scim_dev_client.get_users(filter=filter)
response.to_dict()

{'schemas': ['urn:ietf:params:scim:api:messages:2.0:ListResponse'],
 'totalResults': 3,
 'Resources': [{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
   'meta': {'resourceType': 'User',
    'create': '2024-03-11T19:11:39+00:00',
    'lastModified': '2024-03-11T19:11:39+00:00',
    'location': 'https://api.scim.dev/scim/v2/Users/9b8a54c4-dd1c-48ec-b9eb-c790ec776f02'},
   'id': '9b8a54c4-dd1c-48ec-b9eb-c790ec776f02',
   'active': True,
   'userName': 'rhoward',
   'emails': [{'value': 'ryan.howard@dundermifflin.com',
     'type': 'other',
     'primary': True,
     'display': None},
    {'value': 'ryan.howard@dundermifflin.com',
     'type': 'work',
     'primary': True,
     'display': None}],
   'name': {'formatted': None, 'familyName': 'Howard', 'givenName': 'Ryan'}},
  {'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
   'meta': {'resourceType': 'User',
    'create': '2024-03-11T19:11:39+00:00',
    'lastModified': '2024-03-11T19:11:39+00:00',
    'location': '

Looks like we also should update their emails! Let's do that next.


In [30]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="micheal.scott@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Micheal Scott"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T19:11:37+00:00',
  'lastModified': '2024-03-11T19:16:22+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a54c1-182b-4462-9678-b880db8cf74a'},
 'id': '9b8a54c1-182b-4462-9678-b880db8cf74a',
 'active': False,
 'userName': 'mscott',
 'emails': [{'value': 'micheal.scott@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'micheal.scott@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Scott', 'givenName': 'Micheal'}}

In [31]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="pam.beesly@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Pam Beesly"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T19:11:39+00:00',
  'lastModified': '2024-03-11T19:16:24+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a54c3-f7d7-4096-b07b-83301566dcd2'},
 'id': '9b8a54c3-f7d7-4096-b07b-83301566dcd2',
 'active': True,
 'userName': 'pbeesly',
 'emails': [{'value': 'pam.beesly@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'pam.beesly@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Beesly', 'givenName': 'Pam'}}

In [32]:
from sync_app.scim.patchop import PatchOp, Operation

update_patch_op = PatchOp(
    operations=[
        Operation(
            op="replace", path="emails.value", value="ryan.howard@michealscott.com"
        ),
    ]
)

response = scim_dev_client.update_user(
    id=user_ids["Ryan Howard"],
    ops=update_patch_op,
)

response.to_dict()

{'schemas': ['urn:ietf:params:scim:schemas:core:2.0:User'],
 'meta': {'resourceType': 'User',
  'create': '2024-03-11T19:11:39+00:00',
  'lastModified': '2024-03-11T19:16:26+00:00',
  'location': 'https://api.scim.dev/scim/v2/Users/9b8a54c4-dd1c-48ec-b9eb-c790ec776f02'},
 'id': '9b8a54c4-dd1c-48ec-b9eb-c790ec776f02',
 'active': True,
 'userName': 'rhoward',
 'emails': [{'value': 'ryan.howard@michealscott.com',
   'type': 'other',
   'primary': True,
   'display': None},
  {'value': 'ryan.howard@michealscott.com',
   'type': 'work',
   'primary': True,
   'display': None}],
 'name': {'formatted': None, 'familyName': 'Howard', 'givenName': 'Ryan'}}

WooHoo! Now our job is done! Feel free to try more operition!
